# Using Pyunicore to submit an xml file to, render it and excute parameter sweeps on HPC (JUSUF)

In [1]:
!pip install pyunicore --upgrade
import pyunicore.client as unicore_client
import json
import os
token = clb_oauth.get_token()
tr = unicore_client.Transport(token)
r = unicore_client.Registry(tr, unicore_client._HBP_REGISTRY_URL)
r.site_urls

  Created wheel for pyunicore: filename=pyunicore-0.8.2-py3-none-any.whl size=12948 sha256=e9d8458281c18fe10c90df552e48c876b9c00c9d5559b48d8ba53bedeb40490f
  Stored in directory: /tmp/pip-ephem-wheel-cache-emb_1xbx/wheels/c2/cd/2e/b619a836c33d7eb93fdd32cf6971de221715c6a518ba9dd736
Successfully built pyunicore
  Attempting uninstall: pyunicore
    Found existing installation: pyunicore 0.5.7
    Uninstalling pyunicore-0.5.7:
      Successfully uninstalled pyunicore-0.5.7
You should consider upgrading via the '/opt/app-root/bin/python3 -m pip install --upgrade pip' command.


{'RWTH': 'https://unicore.hpc.itc.rwth-aachen.de:8080/RWTH/rest/core',
 'FZJ_JURECA': 'https://zam2125.zam.kfa-juelich.de:9112/FZJ_JURECA/rest/core',
 'JUWELS': 'https://zam2125.zam.kfa-juelich.de:9112/JUWELS/rest/core',
 'UNICORE-TEST': 'https://catto.cscs.ch:8080/UNICORE-TEST/rest/core',
 'DAINT-CSCS': 'https://brissago.cscs.ch:8080/DAINT-CSCS/rest/core',
 'irene': 'https://hbp-unicore.ccc.cea.fr/irene/rest/core',
 'CINECA-MARCONI': 'https://grid.hpc.cineca.it:9111/CINECA-MARCONI/rest/core',
 'CINECA-GALILEO': 'https://grid.hpc.cineca.it:9111/CINECA-GALILEO/rest/core',
 'JUSUF': 'https://zam2125.zam.kfa-juelich.de:9112/JUSUF/rest/core',
 'BSC-MareNostrum': 'https://unicore-hbp.bsc.es:8080/BSC-MareNostrum/rest/core'}

### Print the XML file

In [2]:
with open("kuramoto_CUDA.xml", "r") as f:
    #print(f.readlines())
    for line in f:
        print (line.rstrip())

<Lems>

    <ComponentType name="Kuramoto"
                   description="Base type of any cell/population which has a (dimensionless) rate _R."
                   value="none">

        <Constant name="omega" domain="" default="60.0 * 2.0 * M_PI_F / 1e3" description="sets the base line frequency for the Kuramoto oscillator in [rad/ms]"/>

        <!-- Parameters will be assumed to be swept. sweep data will be matched by order of entry -->
        <Parameter name="global_speed" dimension='float'/>
        <Parameter name="global_coupling" dimension='float'/>

        <DerivedParameter name="rec_n" expression="1.0f / n_node"/>
        <DerivedParameter name="rec_speed_dt" expression="1.0f / global_speed / (dt)"/>
        <DerivedParameter name="nsig" expression="sqrt(dt) * sqrt(2.0 * 1e-5)"/>

        <Exposure name="V" default="" choices="sin(V)"/>

        <Dynamics>
            <!-- Use PI to select the PI wrapper (ie Kuramoto) -->
            <StateVariable name="V" default="-2, 1"

### Transfer to HPC
Transfer the XML model file to HPC

In [5]:
base_url = "https://zam2125.zam.kfa-juelich.de:9112/JUSUF/rest/core/storages/PROJECT/"
source_path = os.path.join(os.environ['HOME'], "drive/My Libraries/multiscaleteam/Hackathon Sept20/kuramoto_CUDA.xml")

# sites = unicore_client.get_sites(tr)
# client = unicore_client.Client(tr, "https://zam2125.zam.kfa-juelich.de:9112/JUSUF/rest/core/")
# for p in client.get_storages():
#     print (p)

storage = unicore_client.Storage(tr, base_url)
# print(storage.listdir(path))
storage.upload(source_path, destination = "vandervlag1/cuda_only/tvb-hpc/dsl/dsl_cuda/XMLmodels/Kuramoto.xml")

### Job setup

In [60]:
my_job = {}

# executable / application
# arguments for runthingsJusuf are: backend modelname couplings speeds
my_job['Executable'] = "source /p/project/cslns/vandervlag1/cuda_only/tvb-hpc/activate; \
cd /p/project/cslns/vandervlag1/cuda_only/tvb-hpc/dsl/dsl_cuda/example/; \
./runthingsJusuf cuda Kuramoto 120 120;"

# environment vars
# run this on login node, not in batch system
my_job['RunOnLoginNode'] = "true"

my_job['Job type'] = "interactive"

# data stage in - TBD
my_job['Imports'] = []

# data stage out - TBD
my_job['Exports'] = []

# Resources - TBD
my_job['Resources'] = {}

In [61]:
#tr = unicore_client.Transport(token)
HPC_LOC = "https://zam2125.zam.kfa-juelich.de:9112/JUSUF/rest/core"
# HPC_LOC = "https://zam2125.zam.kfa-juelich.de:9112/JUWELS/rest/core"
site = unicore_client.Client(transport=tr,site_url=HPC_LOC)
#print(json.dumps(site.properties, indent=1))

### Submit Job to JUSUF 

In [66]:
job = site.new_job(job_description=my_job)

### Info about JOB

In [68]:
if(job.is_running()):
    print('Job is running')

job.poll()
print('Job is finished')

Job is finished


### Fetch results
Copy the output log from JUSUF to Collab

In [ ]:
remote = storage.stat("vandervlag1/cuda_only/tvb-hpc/dsl/dsl_cuda/example/output.out")
remote.download("output.out")

with open("output.out", "r") as f:
    for line in f:
        print (line.rstrip())

Get .er log

In [342]:
remote = storage.stat("vandervlag1/cuda_only/tvb-hpc/dsl/dsl_cuda/example/error.er")
remote.download("error.er")

with open("error.er", "r") as f:
    for line in f:
        print (line.rstrip())